In [7]:
import pandas as pd
import re
corpus_lemm = pd.read_csv("reviews_score.csv", engine='python')

In [3]:
text = '.'.join(corpus_lemm['review'])
sentences_list = [x.strip() for x in text.split('.')]

### Идея 2

In [4]:
words_sentences_list = [x.split(' ') for x in sentences_list]

In [8]:
from pymystem3 import Mystem
m = Mystem()
def preprocessing(raw_text):
    clean_text = re.sub('\W+', ' ', raw_text) # \W = [^a-zA-Z0-9_]
    return clean_text

for i in range(len(words_sentences_list)):
    for j in range(len(words_sentences_list[i])):
        words_sentences_list[i][j] = preprocessing(words_sentences_list[i][j])

In [9]:
words_sentences_list = [word for word in words_sentences_list if word != ' ']

In [10]:
import gensim
model = gensim.models.Word2Vec(words_sentences_list, size=500, window=10, min_count=2, sg=0)

In [11]:
model

In [12]:
import pickle
pickle.dump(model, open("w2v_model", "wb"))
#model = pickle.load(open("w2v_model", "rb"))

In [13]:
model.wv.save_word2vec_format('model.bin', binary=True)

## пересечение со словарем

In [15]:
with open('sentiment_list.txt','r',encoding='utf-8') as file4:
    new_sent_words = {word.split('\t')[0] for word in file4.readlines()}

In [18]:
with open('rusentilex.csv','r',encoding='utf-8') as file3:
    sentiment_dict = {word.split(',')[0] for word in file3.readlines()}

In [19]:
print('Всего найдено ', len(new_sent_words), ' слов')
print('Пересечение со словарем оценочных выражений: ', len(sentiment_dict & new_sent_words), 'слов, ',
      round(len(sentiment_dict & new_sent_words)/len(new_sent_words), 1)*100, '% списка')

Всего найдено  197  слов
Пересечение со словарем оценочных выражений:  57 слов,  30.0 % списка


In [20]:
for word in sentiment_dict & new_sent_words:
    print(word)

восторженный
гуманный
классный
благодарный
осадок
достойный
двойственный
улыбчивый
обходительный
симпатичный
превосходный
приветливый
отменный
отличный
ужасный
вежливый
доброжелательный
позитивный
дружелюбный
грубить
изумительный
прекрасный
потрясающий
восхитительный
внимательный
предупредительный
великолепный
правильный
шикарный
стильный
отвратительный
неплохой
вкусный
замечательный
приличный
живой
обширный
умеренный
посредственный
качественный
тихий
негативный
удивительный
потрясать
разумный
отрицательный
признательность
оригинальный
демократичный
благодарность
чудесный
противоречивый
улыбаться
испортиться
положительный
душевный
спокойный


In [21]:
for word in (new_sent_words - sentiment_dict):
    print(word)

выступать
молодец 
хороший 
мальчик
фоновый
подстреливать
выражать
бейдж
достойный 
подготовка
разговаривать
обычный
отличный 
ненавязчивый 
плохой 
внимательный 
сытно
отвратительный 
узбекский
изменяться
объяснять
персонал 
вопрос
поблагодарить
бармен
девушка
официанка
громко
пытаться
сервис 
положительный 
японский
воспоминание
замечательный 
менеджер
официант
мероприятие 
спасибо 
вкусно
приятный 
обслуживание 
витражный
хата 
разнообразный 
громкий
купчино
вежливо
умница 
коллектив
вежливый 
гауд
отвечать 
изумительный 
 320р 
басс
милый 
обслуживать 
ребята
орать 
периодически
великолепный 
администратор
оформлять
тетка quot 
грузинский
играть
вопрос 
петь
кавказский
итальянский
суша бар
классный 
средний 
мексикан 
парень
супер 
девушка 
хромать
сытно 
обалденный
марципановый
прекрасный 
официант 
дешевый 
умничек 
задавать
китайский
эмоция 
доброжелательно 
окраина
уединенный
организованный
управляющий
бедненький 
улыбаться 
четко 
девочка
официантка 
впечатление 
пепперони 
но